# 归并排序

Refs:
- https://zhuanlan.zhihu.com/p/124356219

**性能**
- 最好时间复杂度：$O(nlog(n))$
- 最坏时间复杂度：$O(nlog(n))$
- 平均时间复杂度：$O(nlog(n))$
- 空间复杂度：$O(n)$
- 稳定性：✅
- In-place：❌

In [1]:
def merge_sort_recursively(items):
    """归并排序 - 递归实现
    """
    size = len(items)
    temp = [None] * size
    _merge_sort_recursively(items, 0, size - 1, temp)
    return items, []


def _merge_sort_recursively(items, lo, hi, temp):
    """经过该函数处理，items 中 lo~hi 的元素会被排序
    """
    if lo >= hi:
        return

    # 将列表分为 left，right 两部分
    mid = lo + (hi - lo) // 2
    left_lo, left_hi = lo, mid
    right_lo, right_hi = mid + 1, hi

    # 对左右两部分各自递归排序
    _merge_sort_recursively(items, left_lo, left_hi, temp)
    _merge_sort_recursively(items, right_lo, right_hi, temp)

    # 合并左右两部分

    temp_idx = lo
    # 左右两部分列表，采用双指针的方式、将结果合并到临时结果 temp 中
    while left_lo <= left_hi and right_lo <= right_hi:
        if items[left_lo] < items[right_lo]:
            temp[temp_idx] = items[left_lo]
            left_lo += 1   # 左侧部分指针往前移动一个
        else:
            temp[temp_idx] = items[right_lo]
            right_lo += 1  # 右侧部分指针往前移动一个
        temp_idx += 1

    # 上面双指针方式处理完以后，若左侧部分还有剩余
    while left_lo <= left_hi:
        temp[temp_idx] = items[left_lo]
        left_lo += 1
        temp_idx += 1

    # 上面双指针方式处理完以后，若右侧部分还有剩余
    while right_lo <= right_hi:
        temp[temp_idx] = items[right_lo]
        right_lo += 1
        temp_idx += 1

    # 将 temp 中排好序的 set 到原列表中，因为上面的处理过程中，temp_idx 最终会 +1，故需要 -1 才是实际的索引
    temp_idx -= 1
    while temp_idx >= lo:
        items[temp_idx] = temp[temp_idx]
        temp_idx -= 1

In [2]:
%run utils.ipynb

test_sort_funtion(merge_sort_recursively)

In [3]:
def merge_sort_iteratively(items):
    """归并排序 - 迭代法实现
    """
    len_ = len(items)
    temp = [None] * len_

    block_step = 1
    # block_step 每次 *2
    # 下面 【】表示一对，其中分为左侧部分和右侧部分，左侧和右侧会进行归并排序
    # block_step=1 时，列表可以视为【[0,1),[1,2)】【[2,3),[3,4)】...
    # block_step=2 时，列表可以视为【[0,2),[2,4)】【[4,6),[6,8)】...
    # block_step=4 时，列表可以视为【[0,4),[4,8)】【[8,12),[12,16)】...
    # ...
    while block_step < 2 * len_:
        # e.g. len_=7 ==> max(block_step)=6，此时整个列表只有 【[0, len), []】，对应最后一轮归并排序

        # 下面的 while 任务：在给定 block_step 下，将列表中的每一对进行归并排序，e.g. block_step=2
        # 第1对 【[0,2), [2,4)】：将其中的左右两部分进行归并排序
        # 第2对 【[4~6), [6~8)】：将其中的左右两部分进行归并排序
        # 第n对 ...
        block_lo = 0
        while block_lo < len_:  # 循环处理，第一次循环处理上面提到的第一对、第二次循环处理第二对 ...
            # 第n对的 lo, mid, hi
            lo = block_lo
            mid = block_lo + block_step if block_lo + block_step < len_ else len_
            hi = block_lo + 2 * block_step if block_lo + 2 * block_step < len_ else len_

            # 第n对中左边部分和右边部分的 lo, hi
            left_lo, left_hi = lo, mid - 1      # 注意：这儿特意 -1，因为 left_hi 是索引、不能越界
            right_lo, right_hi = mid, hi - 1    # 注意：这儿特意 -1，因为 right_hi 是索引、不能越界

            # 将左边部分和右边部分进行归并排序 ↓↓↓
            temp_idx = left_lo
            while left_lo <= left_hi and right_lo <= right_hi:  # 双指针
                if items[left_lo] < items[right_lo]:  # 若左侧元素更小，将其塞到 temp
                    temp[temp_idx] = items[left_lo]
                    left_lo += 1
                    temp_idx += 1
                else:
                    temp[temp_idx] = items[right_lo]  # 若右侧元素更小，将其塞到 temp
                    right_lo += 1
                    temp_idx += 1
            while left_lo <= left_hi:                 # 上面双指针循环完后，如果左边部分还有剩余
                temp[temp_idx] = items[left_lo]
                left_lo += 1
                temp_idx += 1
            while right_lo <= right_hi:               # 上面双指针循环完后，如果右边部分还有剩余
                temp[temp_idx] = items[right_lo]
                right_lo += 1
                temp_idx += 1
            # 上面处理完之后，第n对中左边部分+右边部分已经有序了 ↑↑↑

            block_lo += 2 * block_step # 处理第 n+1 对
   
        # ======== while end ============================================
        # 上面的 while 循环结束，第1对、第2对、第n对 各自的内部都已经有序了
        # 因为部分有序的结果存在了 temp 中，故将 temp 赋给 items 然后进行下一轮排序
        items, temp = temp, items

        # 然后 double block_step 后，重复上面的步骤
        # double block_step 后
        # 上面的 第1对和第2对 重新组成一对、分为左右两部分、重新进行归并排序
        # 上面的 第2对和第3对 重新组成一对、分为左右两部分、重新进行归并排序
        # ...
        block_step *= 2

    # ============ while end ============================================
    # 内部 while 循环结束后，总会把有序的 temp 赋值给 items，故最终的 items 就是有序的，直接返回即可
    return items, []

In [4]:
%run utils.ipynb

test_sort_funtion(merge_sort_iteratively)